In [38]:
import numpy as np
import pandas as pd
#Here ,every csv file was read under this using "pd.read"
content_types = pd.read_csv("gobe_ContentTypes.csv", sep = "\t")
userFavoriteAsset = pd.read_csv("gobe_UserFavoriteAsset.csv", sep = "\t")
genres = pd.read_csv("gobe_Genres.csv", sep = "\t")
MovieGenreMap = pd.read_csv("gobe_MovieGenreMap.csv", sep = "\t")
ContentRecoMap = pd.read_csv("gobe_ContentRecoMap.csv", sep = "\t")
Content_filtered = pd.read_csv("gobe_Content_filtered.csv", sep = "\t", encoding = "latin-1",low_memory=False)


In [39]:
Movie = pd.read_csv("gobe_Movie.csv", sep = "\t", error_bad_lines = False, encoding = "latin-1")
genres = pd.read_csv("gobe_Genres.csv", sep = "\t")
MovieGenreMap = pd.read_csv("gobe_MovieGenreMap.csv", sep = "\t")
merge=pd.merge(MovieGenreMap,genres[['genre_id','genre_type','asset_type']],on='genre_id',how='left')#Merging of two csv files with extracting whatever factors we want to get from second one and map them using a common value 
#Here onwards more or less "pd.merge" is uses till the end.
merge_mov=pd.merge(Movie,
                 merge[['movie_id','genre_id','genre_type','asset_type']],
                 on='movie_id',how='left')
merge_mov=merge_mov.drop(['description','synopsis','short_description','keywords','creation_time','update_time','eros_rating','startdate','expirydate','cbfc_cert'],axis=1)
merge_mov=merge_mov.drop(['eros_studio_id','critic_review','critic_rating'],axis=1)
merge_mov=merge_mov.loc[merge_mov['asset_type'] == 'MOVIE']

# "AnyDataframe.loc" function helps in locating selected values of specific rows or columns. 

merge.loc[merge['asset_type'] == 'MOVIE']['movie_id'].value_counts()
merge_mov.loc[merge_mov['movie_id'] == 1000074] #Not needed,this line is more for debugging

#"AnyDataframe.drop" function helps in dropping selected columns.
merge_mov1 =merge_mov.drop(["title","release_date","language","rating","release_year","duration","box_office","asset_type"],axis = 1)

# "Pivot" is one important function which helps in assigning row values in a count in columns,a brief read in web will help 
merge_mov2 = merge_mov1.pivot(index='movie_id', columns='genre_type',values='genre_id')

#"AnyDataframe.reset_index" helps in realigning in columns ,this is needed more or less everytime after the pivot is used.

merge_mov_genre_uuid=merge_mov2.reset_index().rename_axis(None, axis=1)
#Once the pivot-table is done,transferred it into a matrix with binary values "0" and "1" using threshold as [0-169] so that since "movie_id" column is also a number ,we dont want that to be 0 or 1.
merge_mov_genre_uuid[(0< merge_mov_genre_uuid)&(merge_mov_genre_uuid<169)]=1

#Fill nullvalues with '0' in next line.
merge_mov_genre_uuid= merge_mov_genre_uuid.fillna(0)

merge_cluster=pd.merge(merge_mov_genre_uuid,
                 merge_mov[['movie_id','language','rating','release_year']],
                 on='movie_id')
merge_cluster=merge_cluster.drop_duplicates(subset=None, keep='first', inplace=False)  

b'Skipping line 883: expected 21 fields, saw 25\nSkipping line 1518: expected 21 fields, saw 25\nSkipping line 5363: expected 21 fields, saw 24\n'


In [3]:
#movie_language pivot table
merge_cluster['values']=1
merge_cluster_lan=merge_cluster.filter(['movie_id','language','values'],axis=1)
merge_cluster_lan = merge_cluster_lan.pivot(index='movie_id', columns='language',values='values').fillna(0)
merge_cluster_lan=merge_cluster_lan.reset_index().rename_axis(None, axis=1)

In [4]:
# This function extracts only language columns mapped to movie_id
#let a movie 'A' be 'hindi',so column 'hin' will be '1' and others will be '0'.
#This is same for other "cluster" dataframes which are about to come.
merge_cluster=pd.merge(merge_cluster,merge_cluster_lan[['movie_id',
 'ben',
 'eng',
 'hin',
 'kan',
 'mal',
 'mar',
 'pan',
 'pun',
 'tam',
 'tel']],on='movie_id',how='left')

In [5]:
#Converted 'release_year' column into 'integer' as they are of different objective type.
# Movies which are released under specific range are brought into one bucket. eg:movies that are released between 1850-1980 will be under a specific column and so on. 
merge_cluster['release_year']=pd.to_numeric(merge_cluster['release_year'],errors='coerce') #(4765829, 21)
merge_cluster['release_year']=pd.cut(merge_cluster['release_year'], [-1,1850,1980,1990,1995,2000,2005,2010,2015,2020], labels=[1850,1980,1990,1995,2000,2005,2010,2015,2020])
merge_cluster['release_year']=pd.to_numeric(merge_cluster['release_year'],errors='coerce') #(4765829, 21)
merge_cluster['values']=1
merge_cluster_year=merge_cluster.filter(['movie_id','release_year','values'],axis=1)
merge_cluster_year = merge_cluster_year.pivot(index='movie_id', columns='release_year',values='values').fillna(0)
merge_cluster_year=merge_cluster_year.reset_index().rename_axis(None, axis=1)

In [6]:
#movie_cluster
movie_cluster=pd.merge(merge_cluster,
                 merge_cluster_year[['movie_id',1850, 1980, 1990, 1995, 2000, 2005, 2010, 2015, 2020]],
                 on='movie_id',how='left').drop(['language','release_year','values'],axis=1)

In [7]:
#Next 2-3 lines are meant for debugging.
Movie[Movie['title'].str.contains("Bajirao")]
# merge_mov.loc[merge_mov['movie_id'] == 1004653]
MovieGenreMap.loc[MovieGenreMap['movie_id'] == 1004653]


,movie_id,genre_id,creation_time,update_time
3623,1004653,7,2016-05-13 10:47:35,2016-05-13 10:47:35
3624,1004653,17,2016-05-13 10:47:35,2016-05-13 10:47:35


In [8]:
### User_content
#The next lines are more or less about user content(history of user from which we are going to bring insights about his/her intrests through their watch history for now) that we have between 
df_s=pd.read_csv("uuid.csv",sep=",",error_bad_lines=False,low_memory=False)
df_s['uuid']=pd.to_numeric(df_s['uuid'],errors='coerce') #(4765829, 21)
df_s=df_s[pd.notnull(df_s['uuid'])]
df_s=df_s.loc[df_s['asset_type'] == 'MOVIE']
df_s=df_s.loc[df_s['content_type_id']==1]

In [9]:
#There are only certain movie-id's which have "movie-id" as well as 'title' ,henceforth we are going to merge them with "user content obtained in previous cell with "movie title" as common column-value.
##User - 5.89 lakh data [55laks user data cut down to 5.89 lakhs till here]
dfs_movieid=pd.merge(df_s,Movie[['title','movie_id']],on='title')
dfs_movieid_filter=pd.merge(dfs_movieid,movie_cluster[['movie_id']],on='movie_id')

In [10]:
dfs_movieid_filter_drop=dfs_movieid_filter.drop(['Unnamed: 0','date',
 'platform',
 'ip',
 'asset_id',
 'content_id',
 'asset_type',
 'country',
 'played_count',
 'total_duration',
 'owner',
 'asset_title',
 'title',
 'content_type_id',
 'user_state',
 'mm_city',
 'mm_country',
 'mm_country_iso_code',
 'mm_subdivisions_iso_code',
 'mm_subdivisions'],axis=1)

In [11]:
dfs_movieid_filter_drop
movie_cluster
user_movieid_pre=pd.merge(dfs_movieid_filter_drop,movie_cluster,on='movie_id')
# user_movieid_pre=user_movieid_pre.drop(['movie_id','rating'],axis=1)

In [12]:
user_movieid_pre.shape

(589246, 39)

In [13]:
user_movieid_pre=user_movieid_pre.groupby('uuid')['Action',
 'Animation',
 'Classics',
 'Comedy',
 'Crime',
 'Devotional',
 'Drama',
 'Family',
 'Horror',
 'Kids',
 'Mature',
 'Mystery',
 'Patriotic',
 'Romance',
 'Romantic Comedy',
 'Sports',
 'Thriller',
 'ben',
 'eng',
 'hin',
 'kan',
 'mal',
 'mar',
 'pan',
 'pun',
 'tam',
 'tel',
 1850,
 1980,
 1990,
 1995,
 2000,
 2005,
 2010,
 2015,
 2020].sum()

In [14]:
#This is one important cell where we have summed up total movies watched by an user into genres,languages and their year of releases.
#For each 

user_movieid_vector=user_movieid_pre.reset_index().rename_axis(None, axis=1)
# user_movieid_vector.iloc[:, 1:18].sum(axis=1)
# user_movieid_vector['uuid']=format("%.11f", 'uuid')
user_movieid_vector['uuid'] = user_movieid_vector['uuid'].astype('int64') #convert scientific format to number
user_movieid_vector['sum']=user_movieid_vector.iloc[:, 28:37].sum(axis=1)
user_movieid_vector=user_movieid_vector[user_movieid_vector['sum']>14] #Had filtered users who ever had watched more than 14 so as to decrease the bias.
user_movieid_vector=user_movieid_vector.drop(['sum'],axis=1) 

In [15]:
u_s=user_movieid_vector.iloc[:, 1:18].sum(axis=1)
###Give user_id of movies to suggest
user_stick=user_movieid_vector.iloc[:, 1:18].div(user_movieid_vector.iloc[:, 1:18].sum(axis=1),axis=0)*100
user_stick=user_stick[user_stick.gt(60).any(axis=1)]
pd.merge(user_movieid_vector.iloc[:, 0:1], user_stick, left_index=True, right_index=True).head(50)

,uuid,Action,Animation,Classics,Comedy,Crime,Devotional,Drama,Family,Horror,Kids,Mature,Mystery,Patriotic,Romance,Romantic Comedy,Sports,Thriller
810,14885147666,6.976744,0.0,0.000000,62.790698,6.976744,0.000000,4.651163,0.000000,0.000000,0.000000,0.0,0.0,0.000000,9.302326,9.302326,0.0,0.000000
2838,15093955238,0.000000,0.0,0.000000,84.210526,0.000000,0.000000,5.263158,0.000000,0.000000,0.000000,0.0,0.0,0.000000,10.526316,0.000000,0.0,0.000000
5342,135534532811,7.407407,0.0,0.000000,70.370370,0.000000,0.000000,0.000000,7.407407,0.000000,0.000000,0.0,0.0,0.000000,7.407407,7.407407,0.0,0.000000
5358,135664274052,21.739130,0.0,0.000000,4.347826,0.000000,0.000000,4.347826,0.000000,0.000000,0.000000,0.0,0.0,0.000000,65.217391,4.347826,0.0,0.000000
5432,136160456175,13.157895,0.0,0.000000,65.789474,0.000000,0.000000,7.894737,0.000000,0.000000,0.000000,0.0,0.0,0.000000,13.157895,0.000000,0.0,0.000000
5532,137565634949,0.000000,0.0,0.000000,66.666667,0.000000,0.000000,14.285714,0.000000,0.000000,0.000000,0.0,0.0,0.000000,9.523810,9.523810,0.0,0.000000
7657,143903096280,14.285714,0.0,0.000000,67.532468,0.000000,0.000000,6.493506,1.298701,0.000000,0.000000,0.0,0.0,1.298701,5.194805,3.896104,0.0,0.000000
7686,143913140048,0.000000,0.0,4.347826,4.347826,0.000000,0.000000,65.217391,0.000000,0.000000,0.000000,0.0,0.0,0.000000,26.086957,0.000000,0.0,0.000000
9101,144916041036,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,100.000000,0.000000,0.0,0.000000
9139,144959579723,0.000000,0.0,0.000000,95.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,5.000000,0.000000,0.0,0.000000


In [65]:
h=1502782492481
# user_movieid_vector.loc[user_movieid_vector['uuid'] == 1500912383114].T
#1524629761295
#1500912383114-my
#1512749664182
#1502782492481
# # user_movieid_vector[user_movieid_vector.uuid == h].index[0]
# # user_movieid_vector.iloc[52471]
# np.sum(np.multiply(user_movieid_vector.loc[user_movieid_vector['uuid'] == 149283764195].values[:,1:18],movie_cluster.iloc[803,1:18].values.flatten().reshape(17,1).T))

In [66]:
#We made sure that User-vector and movie-vector are having same set of columns.ie.,for each user,we'll have columns of count of (each genre he had watched,number of movies in each language as well number of movies in mentioned year of release 
#Now since the movie vector is already in above mentioned order,
#We used "dot product" of each user_vector to all movie_vectors to find the best correlated vectors.
#Since we found 'Genre' is an important factor,we first calculated best genres that suite his/her interest ,[60 best movies that suite genre],then applied same for language,then applied same for years of release.
#In the end heavy weightage had been given to recent years of release movies as suggested by content head.
# i=user_movieid_vector[user_movieid_vector.uuid == h].index[0]
a=user_movieid_vector.loc[user_movieid_vector['uuid'] == h].values[:,1:18]
c=np.zeros([1193, 1])
k=np.zeros([1193,17])
b=np.zeros([1193,17])
m=np.zeros([1193,2])
l=np.zeros([1193,1])
for j in range(1193):
    b[j]=movie_cluster.iloc[j,1:18].values.flatten().reshape(17,1).T
    k[j]=np.multiply(a,b[j])
    c[j][0]=movie_cluster.iloc[j,0]
for j in range(1193):
    l[j][0]=np.sum(k[j])
c=c.reshape(1193,)    
l=l.reshape(1193,)
m=np.vstack((c, l)).T
#genre_based filter
df = pd.DataFrame(m,index=None)
df=df.sort_values(by=[1],ascending=False)
df_genre=df.head(60)
df_genre.columns = ['movie_id', 'score']

In [67]:
score=df_genre.iloc[:,1].values.flatten().reshape(60,1)
# .reshape(10,1).T

df_lan=pd.merge(df_genre,movie_cluster[['movie_id','ben',
 'eng',
 'hin',
 'kan',
 'mal',
 'mar',
 'pan',
 'pun',
 'tam',
 'tel']],on='movie_id').drop(['score'],axis=1)

i=user_movieid_vector[user_movieid_vector.uuid == h].index[0]
a_1=user_movieid_vector.loc[user_movieid_vector['uuid'] == h].values[:,18:28]
c_1=np.zeros([60, 1])
k_1=np.zeros([60,10])
b_1=np.zeros([60,10])
m_1=np.zeros([60,2])
l_1=np.zeros([60,1])
for j in range(60):
    b_1[j]=df_lan.iloc[j,1:11].values.flatten().reshape(10,1).T
    k_1[j]=np.multiply(a_1,b_1[j])
    c_1[j][0]=df_lan.iloc[j,0]
for j in range(60):
    l_1[j][0]=((1.5)*np.sum(k_1[j]))+score[j]
    
c_1=c_1.reshape(60,)
l_1=l_1.reshape(60,)
m_1=np.vstack((c_1, l_1)).T
#genre_based filter
df_1 = pd.DataFrame(m_1,index=None)
df_1=df_1.sort_values(by=[1],ascending=False)
df_genre_lan=df_1.head(40)
df_genre_lan.columns = ['movie_id', 'score_1']
score_1=df_genre.iloc[:,1].values.flatten().reshape(60,1)

In [68]:
#lan_based_filter
print("UUID:",h)
df_years=pd.merge(df_genre_lan,movie_cluster[['movie_id',1850,
 1980,
 1990,
 1995,
 2000,
 2005,
 2010,
 2015,
 2020]],on='movie_id').drop(['score_1'],axis=1)

i=user_movieid_vector[user_movieid_vector.uuid == h].index[0]
a_2=user_movieid_vector.loc[user_movieid_vector['uuid'] == h].values[:,28:37]
c_2=np.zeros([40, 1])
k_2=np.zeros([40,9])
b_2=np.zeros([40,9])
m_2=np.zeros([40,2])
l_2=np.zeros([40,1])
i_2=np.zeros([40,1])

for j in range(40):
    b_2[j]=df_years.iloc[j,1:10].values.flatten().reshape(9,1).T
    k_2[j]=np.multiply(a_2,b_2[j])
    c_2[j][0]=df_genre_lan.iloc[j,0]
    i_2[j][0]=np.sum(df_years.iloc[j,7:10].values.flatten().reshape(3,1).T)*100
for j in range(40):
    l_2[j][0]=np.sum(k_2[j])+(0.5*score[j])+(i_2[j][0])
    
c_2=c_2.reshape(40,)
l_2=l_2.reshape(40,)
m_2=np.vstack((c_2, l_2)).T
df_2 = pd.DataFrame(m_2,index=None)
df_2=df_2.sort_values(by=[1],ascending=False)
df_genre_lan_y=df_2.head(20)
df_genre_lan_y.columns = ['movie_id', 'score_2']

# pd.merge(df_genre_lan_y, dfs_movieid_filter_drop,on='movie_id', how='left', indicator=True)

df_9=dfs_movieid_filter_drop.loc[dfs_movieid_filter_drop['uuid'] ==h]

df_9=df_9.groupby(['uuid', 'movie_id']).size().reset_index(name='count')
df_9=df_9.sort_values(by=['count'],ascending=False)
user_watched_movies=pd.merge(df_9,Movie[['title','movie_id']],on='movie_id')
user_watched_movies.columns = ['uuid', 'movie_id','count','watched in past']

recommended=pd.merge(df_genre_lan_y, user_watched_movies[['movie_id']],on='movie_id', how='left', indicator=True).query("_merge == 'left_only'")
recommended=pd.merge(recommended,Movie[['title','movie_id','rating']],on='movie_id').drop(['_merge'],axis=1)
recommended=recommended[['movie_id','title','score_2','rating']]
recommended.columns = ['movie_id_1', 'recommended','score_2','rating']
recommended['rating']=recommended['rating'].astype('float64')
recommended['score_3']=recommended['score_2']+10*recommended['rating']
recommended_1=recommended.sort_values(by=['score_3'],ascending=False).drop(['score_2'],axis=1)
# Final_recommendation = pd.concat([user_watched_movies.reset_index(drop=True), recommended_1], axis=1)
# pd.concat([recommended_1.reset_index(drop=True), user_watched_movies], axis=1)
Final_recommendation = pd.concat([recommended_1.reset_index(drop=True), user_watched_movies], axis=1)
Final_recommendation=Final_recommendation.replace(np.NaN,'-')
Final_recommendation=Final_recommendation[['uuid','movie_id','count','watched in past','movie_id_1','recommended','rating','score_3']]
from IPython.display import HTML
HTML(Final_recommendation.to_html(classes='table table-bordered '))
# uuid1result = Final_recommendation
uuid4result = Final_recommendation

UUID: 1502782492481


In [75]:
print("UUID:1512749664182 ")
uuid1result

UUID:1512749664182 


,uuid,movie_id,count,watched in past,movie_id_1,recommended,rating,score_3
0,1.51275e+12,1.00022e+06,18,Golmaal 3,1000240.0,Heyy Babyy,4.0,175.0
1,1.51275e+12,1.00047e+06,3,Partner,1000493.0,Pyaar Ke Side Effect,3.5,170.0
2,1.51275e+12,1.00025e+06,1,Hum Dil De Chuke Sanam,1005020.0,Sparshaa,3.0,165.0
3,-,-,-,-,1000585.0,Ugly Aur Pagli,2.5,160.0
4,-,-,-,-,1004650.0,Drishyam,4.5,159.0
5,-,-,-,-,1000429.0,Mr. Hot Mr. Kool,2.0,155.0
6,-,-,-,-,1005216.0,Mooru Manasu,2.0,154.0
7,-,-,-,-,1000633.0,Desi Boyz,4.0,154.0
8,-,-,-,-,1000636.0,Ready,4.0,154.0
9,-,-,-,-,1004947.0,Om Shakthi,0.0,114.0


In [74]:
print("UUID:1500912383114 ")
uuid2result

UUID:1500912383114 


,uuid,movie_id,count,watched in past,movie_id_1,recommended,rating,score_3
0,1.50091e+12,1.00001e+06,13,Aa Dekhen Zara,1000457.0,Om Shanti Om,4.0,162.5
1,1.50091e+12,1.00004e+06,2,Aladin,1000331.0,Karthik Calling Karthik,4.0,161.5
2,-,-,-,-,1000103.0,Bhool Bhulaiyaa,4.0,161.5
3,-,-,-,-,1000356.0,Knock Out,3.5,157.5
4,-,-,-,-,1000183.0,Dosh,3.5,156.5
5,-,-,-,-,1000454.0,No Smoking,3.0,152.5
6,-,-,-,-,1000142.0,Dahek - A Restless Mind,3.0,152.5
7,-,-,-,-,1000262.0,I See You,3.0,151.5
8,-,-,-,-,1000113.0,Bolo Raam,3.0,151.5
9,-,-,-,-,1005019.0,Sararat,3.0,151.5


In [76]:
print("UUID:1524629761295 ")
uuid3result

UUID:1524629761295 


,uuid,movie_id,count,watched in past,movie_id_1,recommended,rating,score_3
0,1.52463e+12,1.00465e+06,9,Bajirao Mastani - Tamil,1004907.0,Naan Sivanagiren,3.0,163.5
1,1.52463e+12,1.00465e+06,9,Bajirao Mastani - Telugu,1000583.0,U Me Aur Hum,4.0,155.0
2,1.52463e+12,1.00016e+06,1,Devdas,1000525.0,Salaam-e-Ishq,3.5,150.0
3,1.52463e+12,1.00021e+06,1,Ghazab,1000197.0,Ek Vivah Aisa Bhi,3.0,145.0
4,-,-,-,-,1000599.0,Woh 7 Din,4.5,61.5
5,-,-,-,-,1000573.0,Tere Naam,4.0,56.0
6,-,-,-,-,1004933.0,Wanted,3.5,51.5
7,-,-,-,-,1000538.0,Shabd,3.5,51.0
8,-,-,-,-,1000259.0,Hum Tumhare Hain Sanam,3.5,51.0
9,-,-,-,-,1000091.0,Bewafaa,3.0,46.0


In [77]:
print("UUID:1502782492481 ")
uuid4result

UUID:1502782492481 


,uuid,movie_id,count,watched in past,movie_id_1,recommended,rating,score_3
0,1.50278e+12,1.00025e+06,9,Hum Dil De Chuke Sanam,1004653.0,Bajirao Mastani - Tamil,5.0,162.0
1,1.50278e+12,1.00052e+06,6,Saajan,1000583.0,U Me Aur Hum,4.0,152.0
2,-,-,-,-,1000637.0,Rockstar,4.0,152.0
3,-,-,-,-,1000587.0,Umrao Jaan,3.5,147.0
4,-,-,-,-,1000197.0,Ek Vivah Aisa Bhi,3.0,142.0
5,-,-,-,-,1000590.0,Vaada Raha - I Promise,3.0,142.0
6,-,-,-,-,1000420.0,Milenge Milenge,3.0,142.0
7,-,-,-,-,1000593.0,Veer,3.0,142.0
8,-,-,-,-,1005164.0,Saawariya - Khatu Shyam Ji Ki Amar Gatha,2.5,137.0
9,-,-,-,-,1005107.0,Thavarina Aase,2.5,137.0
